In [ ]:
import pandas as pd
from tqdm import tqdm
from signal_metadata_extraction import MetadataExtractor

In [ ]:
metadata_extractor = MetadataExtractor()

In [ ]:
df_divExplorer = pd.read_csv('fearless_steps_apollo/ASR_track2_dev.csv')

In [ ]:
metadatas = []
audio_col = 'id'

for i in tqdm(range(len(df_divExplorer))):

    audio_id = df_divExplorer[audio_col].iloc[i]
    audio_file = 'fearless_steps_apollo/FSC_P3_Train_Dev/Audio/Segments/ASR_track2/Dev/' + audio_id + '.wav'
    signal_metadata = metadata_extractor.signal_metadata(audio_file)
    
    sentence = df_divExplorer['text'].iloc[i] 
    text_metadata = metadata_extractor.text_metadata(sentence)
    mixed_metadata = metadata_extractor.mixed_metadata(signal_metadata, text_metadata)
    metadatas.append(signal_metadata + text_metadata + mixed_metadata)

In [ ]:
meta_cols = metadata_extractor.list_signal_metadata \
        + metadata_extractor.list_text_metadata \
        + metadata_extractor.list_sig_text_metadata
df_divExplorer_metas = pd.concat([df_divExplorer, pd.DataFrame(metadatas, columns=meta_cols)], axis=1)

print("Metadata columns: ", meta_cols)
print("---")
print("Df for DivExplorer Columns: ", df_divExplorer_metas.columns)

In [ ]:
snrs = []
spectral_flatnesses = []
audio_col = 'id'

for i in tqdm(range(len(df_divExplorer_metas))):

    audio_id = df_divExplorer[audio_col].iloc[i]
    audio_file = 'fearless_steps_apollo/FSC_P3_Train_Dev/Audio/Segments/ASR_track2/Dev/' + audio_id + '.wav'

    ## SNR
    import librosa
    signal, sr = librosa.load(audio_file, sr=8_000)
    rms = librosa.feature.rms(y=signal)
    snr = 10 * np.log10(np.mean(rms ** 2) / np.mean(signal ** 2))

    ## Spectral Flatness
    spectral_flatness = librosa.feature.spectral_flatness(y=signal)
    spectral_flatness = 10 * np.log10(spectral_flatness)
    spectral_flatness = np.mean(spectral_flatness)

    snrs.append(snr)
    spectral_flatnesses.append(spectral_flatness)

In [ ]:
df_divExplorer_metas = pd.concat([df_divExplorer_metas, pd.DataFrame(snrs, columns=['snr'])], axis=1)
df_divExplorer_metas = pd.concat([df_divExplorer_metas, pd.DataFrame(spectral_flatnesses, columns=['spectral_flatness'])], axis=1)

In [ ]:
df_divExplorer_metas.to_csv('fearless_steps_apollo/ASR_track2_dev_metadata.csv', index=False)